In [62]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import pprint
import csv
import glob
import os

In [63]:
diab_hosp_pd = pd.read_csv('Resources/hospitalization.csv') #TRACT_FIPS
diab_hosp_pd = diab_hosp_pd.rename(columns={"TRACT_FIPS": "FIPS"})
diab_adults_pd = pd.read_csv('Resources/inadults.csv') #Census_Tract_FIPS
diab_adults_pd = diab_adults_pd.rename(columns={"Census_Tract_FIPS": "FIPS"})
diab_income_pd = pd.read_csv('Resources/income.csv') #FIPS
diab_food_pd = pd.read_csv('Resources/foodenviro.csv') #FIPS


merged_diabetes_df = pd.merge(diab_hosp_pd, diab_adults_pd, on = "FIPS")
merged_diabetes_df = pd.merge(merged_diabetes_df, diab_income_pd, on = "FIPS")
merged_diabetes_df = pd.merge(merged_diabetes_df, diab_food_pd, on = "FIPS")
merged_diabetes_df = merged_diabetes_df.dropna()
merged_diabetes_df.to_csv('Resources/merged.csv')

In [64]:
merged_diabetes_rural = merged_diabetes_df[merged_diabetes_df["Rural"]== 1]
ct_rural_count = merged_diabetes_rural.count()
print(ct_rural_count)

OBJECTID_x          264
FIPS                264
TRACT_NAME          264
COUNTY_NAME         264
DIABETES_ADJRATE    264
                   ... 
lakids20share       264
laseniors20         264
laseniors20share    264
lahunv20            264
lahunv20share       264
Length: 87, dtype: int64


In [65]:
merged_diabetes_urban = merged_diabetes_df[merged_diabetes_df["Urban"]== 1]
ct_urban_count = merged_diabetes_urban.count()
print(ct_urban_count)

OBJECTID_x          964
FIPS                964
TRACT_NAME          964
COUNTY_NAME         964
DIABETES_ADJRATE    964
                   ... 
lakids20share       964
laseniors20         964
laseniors20share    964
lahunv20            964
lahunv20share       964
Length: 87, dtype: int64
